In [80]:
import pandas as pd

raw_df = pd.read_csv('merged.csv')
filtered_df = raw_df[raw_df['predTag'] == 1]
filtered_df.to_csv('filtered_predTag_1.csv',index=False)

In [81]:
def get_IOU(x1, y1, width1, height1, x2, y2, width2, height2):

    x1_min = x1 - width1/2
    y1_min = y1 - height1/2
    x1_max = x1 + width1/2
    y1_max = y1 + height1/2
    
    x2_min = x2 - width2/2
    y2_min = y2 - height2/2
    x2_max = x2 + width2/2
    y2_max = y2 + height2/2


    if x1_min > x2_min:
        xmin_temp = x1_min
        ymin_temp = y1_min
        xmax_temp = x1_max
        ymax_temp = y1_max

        x1_min = x2_min
        y1_min = y2_min
        x1_max = x2_max
        y1_max = y2_max

        x2_min = xmin_temp
        y2_min = ymin_temp
        x2_max = xmax_temp
        y2_max = ymax_temp 
    
    if x1_max > x2_min:
        if x2_max <= x1_max:
            if  (y2_max <=y1_min) or (y2_min >= y1_max):
                return False, 0
                
            elif (y2_max >= y1_max) and (y2_min >= y1_min):
                w = x2_max - x1_min
                h = y1_max - y2_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)
                
            elif (y2_max >= y1_max) and (y2_min < y1_min):
                w = x2_max - x1_min
                h = y1_max - y1_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)
                
            elif (y2_max< y1_max) and (y2_min>=y1_min):
                w = x2_max - x2_min
                h = y2_max - y2_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)
            elif (y2_max < y1_max) and (y2_min< y1_min):
                w = x2_max - x2_min
                h = y2_max - y1_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)  
        else:
            if (y2_max <=y1_min) or (y2_min >= y1_max):
                return False, 0

            elif (y2_max >= y1_max) and (y2_min >= y1_min):
                w = x1_max - x2_min
                h = y1_max - y2_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)

            elif (y2_max >= y1_max) and (y2_min < y1_min):
                w = x1_max - x2_min
                h = y1_max- y1_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)

            elif (y2_max< y1_max) and (y2_min>=y1_min):
                w = x1_max - x2_min
                h = y2_max - y2_min
                return True, w*h/(width1*height1 + width2*height2 - w*h)
                
            elif (y2_max < y1_max) and (y2_min< y1_min):
                w = x1_max - x2_min
                h = y2_max - y1_min
                return True, w*h/(width1*height1 + width2*height2 - w*h) 
            
    
    else:
        return False, 0

In [82]:
gt_df = pd.read_csv('gt.csv')
filtered_df2 = pd.read_csv('filtered_predTag_1.csv')
filtered_df = filtered_df2
filtered_df = filtered_df[filtered_df['labelID'] != 9]
# filtered_df2.to_csv('filtered_predTag_2.csv')
print(filtered_df['ImageID'])

0          0
1          1
2          1
3          1
4          2
        ... 
5355    3402
5357    3404
5359    3405
5362    3406
5364    3407
Name: ImageID, Length: 3645, dtype: int64


In [83]:


overlap_list = []

# 获取共同的ImageID
common_image_ids = set(gt_df['ImageID']).intersection(set(filtered_df['ImageID']))
print(common_image_ids)

{0, 1, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 115, 152, 153, 154, 155, 156, 158, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 278, 282, 287, 288, 29

In [84]:
# 读取CSV文件

# 遍历每个共同的ImageID
for image_id in common_image_ids:
    gt_data = gt_df[gt_df['ImageID'] == image_id]
    filtered_data = filtered_df[filtered_df['ImageID'] == image_id]

    # 遍历每个gt中的矩形
    for _, gt_row in gt_data.iterrows():
        gt_info = gt_row[['ImageID', 'labelID', 'boxID', 'x center', 'y center', 'width', 'height']]

        for _, filtered_row in filtered_data.iterrows():
            if filtered_row["labelID"]==9:
                continue
            filtered_info = filtered_row[['ImageID', 'labelID','boxID','predTag', 'x center', 'y center', 'width', 'height']]


            # 计算重叠区域的面积
            _, overlap_area = get_IOU(gt_info['x center'], gt_info['y center'], gt_info['width'], gt_info['height'], \
                                   filtered_info['x center'], filtered_info['y center'], filtered_info['width'], filtered_info['height'] )
           

            overlap_list.append({
                'ImageID': image_id,
                'labelID_gt': gt_info['labelID'],
                'labelID_p': filtered_info['labelID'],
                'boxID_gt': gt_info['boxID'],
                'boxID_p': filtered_info['boxID'],
                'predTag_p': filtered_info['predTag'],
                'x_center_gt': gt_info['x center'],
                'y_center_gt': gt_info['y center'],
                'width_gt': gt_info['width'],
                'height_gt': gt_info['height'],             
                'x_center_p': filtered_info['x center'],
                'y_center_p': filtered_info['y center'],
                'width_p': filtered_info['width'],
                'height_p': filtered_info['height'],
                'Overlap': overlap_area
            })

# 转换为DataFrame并保存结果
overlap_df = pd.DataFrame(overlap_list)
overlap_df.to_csv('overlap.csv', index=False)
print(common_image_ids)



{0, 1, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 115, 152, 153, 154, 155, 156, 158, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 275, 278, 282, 287, 288, 29

In [85]:
i = 0
j = 0
overlap_df['判定'] = 'OK'
for (image_id, label_id), group in overlap_df.groupby(['ImageID', 'labelID_p']):
    if all(group['Overlap'] == 0):
        overlap_df.loc[(overlap_df['ImageID'] == image_id) & (overlap_df['labelID_p'] == label_id), '判定'] = '过杀'
        j=j+1
for (image_id, label_id), group in overlap_df.groupby(['ImageID', 'labelID_p']):
    if (group['Overlap'] != 0).any():
        
        # if((group['Overlap'] != 0).sum() > 1)
        max_overlap_idx = group['Overlap'].idxmax()
        label_gt = group.loc[max_overlap_idx, 'labelID_gt']
        label_p = group.loc[max_overlap_idx, 'labelID_p']
        # if(image_id,label_id)==(1105,1):
        #     print(box_id_gt)
        #     print(box_id_p)
        #     print("ok")
        if(label_gt == label_p ):
            overlap_df.loc[(overlap_df['ImageID'] == image_id) & (overlap_df['labelID_p'] == label_id), '判定'] = '一致NG'
        if((label_gt!= label_p) and (label_gt !=9)):
            overlap_df.loc[(overlap_df['ImageID'] == image_id) & (overlap_df['labelID_p'] == label_id), '判定'] = '错杀'
        if(label_gt == 9):
            print(label_p,"fine")
            overlap_df.loc[(overlap_df['ImageID'] == image_id) & (overlap_df['labelID_p'] == label_id), '判定'] = '过杀'
            i= i+1


overlap_df.drop(columns=['boxID_gt', 'Overlap','x_center_gt','y_center_gt', 'width_gt','height_gt'], inplace=True)

# 合并相同 ImageID 和 boxID_p 的行
pre = overlap_df.groupby(['ImageID', 'boxID_p'], as_index=False).first()

pre.to_csv('results_pre.csv', index=False)
# print(i)
# print(j)
# print("good")


5 fine
7 fine
12 fine
12 fine
7 fine
7 fine
12 fine
7 fine
0 fine
7 fine
0 fine
7 fine
8 fine
12 fine
0 fine
12 fine
7 fine
12 fine
12 fine
12 fine
12 fine
7 fine
12 fine
12 fine
12 fine
12 fine
12 fine
12 fine
12 fine
0 fine
7 fine
0 fine
7 fine
7 fine
12 fine
7 fine
0 fine
7 fine
7 fine
7 fine
0 fine
7 fine
7 fine
0 fine
0 fine
7 fine
7 fine
7 fine
7 fine
7 fine
0 fine
7 fine
7 fine
7 fine


In [86]:
common_image_ids = set(gt_df['ImageID']).intersection(set(filtered_df2['ImageID']))
print(common_image_ids)
overlap_list2 = []
for image_id in common_image_ids:
    gt_data = gt_df[gt_df['ImageID'] == image_id]
    filtered_data = filtered_df2[filtered_df2['ImageID'] == image_id]

    # print(gt_data)
    # print("*****")
    # print(filtered_data['ImageID'])

    # 遍历每个gt中的矩形
    for _, gt_row in gt_data.iterrows():
        gt_info = gt_row[['ImageID', 'labelID', 'boxID', 'x center', 'y center', 'width', 'height']]

        for _, filtered_row in filtered_data.iterrows():
            
            filtered_info = filtered_row[['ImageID', 'labelID','boxID','predTag', 'x center', 'y center', 'width', 'height']]


            # 计算重叠区域的面积
            _, overlap_area = get_IOU(gt_info['x center'], gt_info['y center'], gt_info['width'], gt_info['height'], \
                                   filtered_info['x center'], filtered_info['y center'], filtered_info['width'], filtered_info['height'] )
           

            overlap_list2.append({
                'ImageID': image_id,
                'labelID_gt': gt_info['labelID'],
                'labelID_p': filtered_info['labelID'],
                'boxID_gt': gt_info['boxID'],
                'boxID_p': filtered_info['boxID'],
                'predTag_p': filtered_info['predTag'],
                'x_center_gt': gt_info['x center'],
                'y_center_gt': gt_info['y center'],
                'width_gt': gt_info['width'],
                'height_gt': gt_info['height'],             
                'x_center_p': filtered_info['x center'],
                'y_center_p': filtered_info['y center'],
                'width_p': filtered_info['width'],
                'height_p': filtered_info['height'],
                'Overlap': overlap_area
            })

# 转换为DataFrame并保存结果
overlap_df2 = pd.DataFrame(overlap_list2)
overlap_df2.to_csv('overlap2.csv', index=False)

{0, 1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224,

In [87]:
df = pd.read_csv('overlap2.csv')
# print(df)
df['判定'] = 'OK'
for (image_id, label_id), group in df.groupby(['ImageID', 'labelID_gt']):
    # 检查所有 Overlap 是否都为 0
    all_overlap_zero = all(group['Overlap'] == 0)
    
    # 检查所有 Overlap 不为 0 的行的 labelID_p 是否都等于 9
    non_zero_overlap_all_nine = all(group.loc[group['Overlap'] != 0, 'labelID_p'] == 9)
    
    if all_overlap_zero or non_zero_overlap_all_nine:
        print(image_id)
        # 为 9 的那行判定为一致pass，不为 9 的那行判定为漏杀
        df.loc[(df['ImageID'] == image_id) & (df['labelID_gt'] == label_id) & (df['labelID_gt'] == 9), '判定'] = '一致pass'
        df.loc[(df['ImageID'] == image_id) & (df['labelID_gt'] == label_id) & (df['labelID_gt'] != 9), '判定'] = '漏杀'
    # else:
    #     # if((group['Overlap'] != 0).sum() > 1)
    #     max_overlap_idx = group['Overlap'].idxmax()
    #     box_id_gt = group.loc[max_overlap_idx, 'boxID_gt']
    #     box_id_p = group.loc[max_overlap_idx, 'boxID_p']
        
    #     if(box_id_gt == box_id_p ):
    #         # print(112)
    #         df.loc[(df['ImageID'] == image_id) & (df['labelID_gt'] == label_id), '判定'] = '一致NG'
    #     if((box_id_gt != box_id_p) and (box_id_gt !=9)):
    #         # print(223)
    #         df.loc[(df['ImageID'] == image_id) & (df['labelID_gt'] == label_id), '判定'] = '错杀'
    #     if(box_id_gt == 9):
    #         # print(334)
    #         df.loc[(df['ImageID'] == image_id) & (df['labelID_gt'] == label_id), '判定'] = '过杀'


df.drop(columns=['boxID_p', 'Overlap','x_center_p','y_center_p', 'width_p','height_p' ], inplace=True)
df = df[df['判定'] != 'OK']

# 合并相同 ImageID 和 boxID_p 的行
gt = df.groupby(['ImageID', 'boxID_gt'], as_index=False).first()
gt.to_csv('results_gt.csv', index=False)


9
108
109
110
111
112
113
114
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
157
159
160
161
162
163
187
188
223
225
256
274
276
277
279
280
281
283
284
285
286
289
292
293
338
445
449
450
451
452
453
454
460
462
463
464
465
488
489
507
508
511
512
513
514
522
594
595
598
600
609
619
620
621
624
625
626
627
630
631
632
634
642
642
653
656
656
658
701
703
704
705
706
707
708
709
721
722
723
724
725
726
727
728
729
731
732
733
734
735
736
737
738
739
740
743
755
787
799
800
800
809
819
822
824
825
826
827
836
846
847
851
852
853
854
855
856
858
861
862
863
864
865
866
868
869
870
872
873
875
876
877
878
879
880
881
883
884
885
886
888
892
893
894
895
896
897
898
899
900
901
902
903
904
906
907
908
909
911
912
913
914
917
918
920
921
922
923
924
925
926
927
929
930
931
932
933
940
941
942
944
950
951
952
964
965
970
972
973
980
981
1067
1076
1077
1080
1082
1084
1086
1087
1092
1096
1100
1103
11

In [88]:
# df = pd.read_csv('overlap_result.csv')

# # 定义判定逻辑的函数
# def judge_row(row):
#     if row['Overlap'] == 0:
#         return 'overkill'
#     if row['labelID_gt'] == 9:
#         if row['labelID_p'] == 9:
#             return 'same pass'
#         else:
#             return 'overkill'
#     else:
#         if row['labelID_p'] == 9:
#             return 'missing kill'
#         elif row['labelID_p'] == row['labelID_gt']:
#             return 'same NG'
#         else:
#             return 'wrong kill'

# # 应用判定逻辑，创建新列
# df['判定'] = df.apply(judge_row, axis=1)

# # 保存新的CSV文件
# df.to_csv('results_bybox.csv', index=False)

# 统计部分

In [89]:
# 计算各类判定的比率

same_ng_count = len(pre[pre['判定'] == '一致NG'])+len(gt[gt['判定'] == '一致NG'])
same_pass_count = len(pre[pre['判定'] == '一致pass'])+len(gt[gt['判定'] == '一致pass'])
over_kill_count = len(pre[pre['判定'] == '过杀'])+len(gt[gt['判定'] == '过杀'])
wrong_kill_count = len(pre[pre['判定'] == '错杀'])+len(gt[gt['判定'] == '错杀'])
miss_kill_count = len(pre[pre['判定'] == '漏杀'])+len(gt[gt['判定'] == '漏杀'])
total_count = same_ng_count +same_pass_count+over_kill_count+wrong_kill_count+miss_kill_count 
same_ng_rate = same_ng_count / total_count
same_pass_rate = same_pass_count / total_count
over_kill_rate = over_kill_count / total_count
wrong_kill_rate = wrong_kill_count / total_count
miss_kill_rate = miss_kill_count / total_count

print(f"一致NG: {same_ng_count:.2%}")
print(f"一致通过: {same_pass_count:.2%}")
print(f"过杀: {over_kill_count:.2%}")
print(f"错杀: {wrong_kill_count:.2%}")
print(f"漏杀: {miss_kill_count:.2%}")

# 打印比率结果
print(f"一致NG率: {same_ng_rate:.2%}")
print(f"一致通过率: {same_pass_rate:.2%}")
print(f"过杀率: {over_kill_rate:.2%}")
print(f"错杀率: {wrong_kill_rate:.2%}")
print(f"漏杀率: {miss_kill_rate:.2%}")

# 将比率信息存入DataFrame
rate_data = {
    '比率类别': ['一致NG率', '一致通过率', '过杀率', '错杀率', '漏杀率'],
    '比率': [same_ng_rate, same_pass_rate, over_kill_rate, wrong_kill_rate, miss_kill_rate]
}
rate_df = pd.DataFrame(rate_data)

# 保存比率信息到单独的CSV文件
rate_df.to_csv('porbabilities_bybox.csv', index=False, encoding='utf-8-sig')

一致NG: 260500.00%
一致通过: 102900.00%
过杀: 6200.00%
错杀: 97800.00%
漏杀: 1100.00%
一致NG率: 55.60%
一致通过率: 21.96%
过杀率: 1.32%
错杀率: 20.88%
漏杀率: 0.23%
